In [1]:
import json
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from googletrans import Translator
from langdetect import detect
import numpy as np



In [2]:
df = pd.read_csv("Dataframe_Wrangled_NoDuplicates.csv")
df.head()

,id,conversations
0,Og9h3C1,"[{'from': 'human', 'value': ""root@openvpn:/hom..."
1,A5AbcES,"[{'from': 'human', 'value': 'In Java, I want t..."
2,IWkMGRK,"[{'from': 'human', 'value': 'I have the follow..."
3,HskFS9I,"[{'from': 'human', 'value': '1 / 1como crear u..."
4,yn2eWCt,"[{'from': 'human', 'value': 'how do I add mult..."


In [3]:
print(len(df))

23277


In [4]:
# parse conversations
text_list = []
text_arrays = []
for conversation in df["conversations"]:
    text = ""
    text_arr = []
    parsed_conversation = eval(conversation)
    for dialogue in parsed_conversation:
        text += dialogue["value"] + " "
        text_arr.append(dialogue["value"])
    text_list.append(text)
    text_arrays.append(text_arr)

# create column with the conversation text
df["text"] = text_list
df["values"] = text_arrays
        


In [8]:
df.head()


,id,conversations,text,values
0,Og9h3C1,"[{'from': 'human', 'value': ""root@openvpn:/hom...",root@openvpn:/home/openvpn# ./openvpn-install....,[root@openvpn:/home/openvpn# ./openvpn-install...
1,A5AbcES,"[{'from': 'human', 'value': 'In Java, I want t...","In Java, I want to replace string like ""This i...","[In Java, I want to replace string like ""This ..."
2,IWkMGRK,"[{'from': 'human', 'value': 'I have the follow...",I have the following C++ function: \nvoid add_...,[I have the following C++ function: \nvoid add...
3,HskFS9I,"[{'from': 'human', 'value': '1 / 1como crear u...",1 / 1como crear un rol para kuberntes y dar so...,[1 / 1como crear un rol para kuberntes y dar s...
4,yn2eWCt,"[{'from': 'human', 'value': 'how do I add mult...",how do I add multiple new columns in m for pow...,[how do I add multiple new columns in m for po...


In [11]:
# determine language of each conversation
languages = []
for value in df["values"]:
    try:
        lang = detect(value[0])
        languages.append(lang)
    except:
        try:
            lang = detect(value[1])
            languages.append(lang)
        except:
            print("error2")
            print(value[1])


df["lang"] = languages


In [12]:
df.head()

,id,conversations,text,values,lang
0,Og9h3C1,"[{'from': 'human', 'value': ""root@openvpn:/hom...",root@openvpn:/home/openvpn# ./openvpn-install....,[root@openvpn:/home/openvpn# ./openvpn-install...,en
1,A5AbcES,"[{'from': 'human', 'value': 'In Java, I want t...","In Java, I want to replace string like ""This i...","[In Java, I want to replace string like ""This ...",en
2,IWkMGRK,"[{'from': 'human', 'value': 'I have the follow...",I have the following C++ function: \nvoid add_...,[I have the following C++ function: \nvoid add...,en
3,HskFS9I,"[{'from': 'human', 'value': '1 / 1como crear u...",1 / 1como crear un rol para kuberntes y dar so...,[1 / 1como crear un rol para kuberntes y dar s...,es
4,yn2eWCt,"[{'from': 'human', 'value': 'how do I add mult...",how do I add multiple new columns in m for pow...,[how do I add multiple new columns in m for po...,en


In [ ]:
df_shortened = df[['id', 'values', 'lang']]
df_shortened.to_csv('file_name.csv', index=False, encoding='utf-8')


In [13]:
df["lang"].value_counts()

lang
en       18359
ko         827
zh-cn      618
fr         533
es         455
ja         242
ca         201
it         179
pt         166
vi         160
de         148
no         126
ru         125
nl         125
ro         117
sw          89
et          79
zh-tw       74
af          61
fi          60
cy          58
id          58
pl          54
da          53
tl          41
sv          37
tr          30
so          28
hr          28
sk          23
sl          20
uk          19
th          16
cs          13
lt          11
he           9
bg           8
hu           7
sq           6
mk           5
fa           3
lv           2
ar           2
el           2
Name: count, dtype: int64

In [36]:
df['translated_column_test'] = np.nan

In [14]:
translator = Translator()

async def translate(value):
      result = await translator.translate(value, dest="en")
      return result


async def repeat_until_success(action, text, max_attempts=float('inf')):
    attempts = 0
    while attempts < max_attempts:
        try:
            result = await action(text)
            return result  # Successful execution, return the result
        except Exception as e:
            attempts += 1
            print(f"Attempt {attempts} failed: {e}")
    print(f"Action failed after {max_attempts} attempts.")
    return None # or raise an exception if appropriate

translated_values = []
for index, item in df["values"].items():
    if df["lang"][index] != "en":
        translated_conversation = []
        for element in item:
            # a=re.compile("<.*?>")
            # element = a.sub('', element)
            if len(element) > 15000:
                chunks = []
                for i in range(0, len(element), 15000):
                    chunks.append(element[i:i + 15000])
                
                translated_chunks = []
                for chunk in chunks:
                    # result = await translate(chunk)
                    result = await repeat_until_success(translate, chunk)
                    if result is not None:
                        translated_chunks.append(result.text)
                    else: 
                        print("error")
                
                result = " ".join(translated_chunks)
                translated_conversation.append(result)

            else:
                # result = await translate(element)
                result = await repeat_until_success(translate, element)
                if result is not None:
                    translated_conversation.append(result.text)
                else:
                    print("error")
                
        translated_values.append(translated_conversation)
    else:
        translated_values.append(item)

df["translated_values"] = translated_values




Attempt 1 failed: Invalid input ConnectionInputs.RECV_PING in state ConnectionState.CLOSED
Attempt 1 failed: 
Attempt 1 failed: 
Attempt 2 failed: 


CancelledError: 

In [39]:
df.head()

,id,conversations,text,values,lang,translated_column_test
0,Og9h3C1,"[{'from': 'human', 'value': ""root@openvpn:/hom...",root@openvpn:/home/openvpn# ./openvpn-install....,[root@openvpn:/home/openvpn# ./openvpn-install...,en,[root@openvpn:/home/openvpn# ./openvpn-install...
1,A5AbcES,"[{'from': 'human', 'value': 'In Java, I want t...","In Java, I want to replace string like ""This i...","[In Java, I want to replace string like ""This ...",en,"[In Java, I want to replace string like ""This ..."
2,IWkMGRK,"[{'from': 'human', 'value': 'I have the follow...",I have the following C++ function: \nvoid add_...,[I have the following C++ function: \nvoid add...,en,[I have the following C++ function: \nvoid add...
3,HskFS9I,"[{'from': 'human', 'value': '1 / 1como crear u...",1 / 1como crear un rol para kuberntes y dar so...,[1 / 1como crear un rol para kuberntes y dar s...,es,[1 /1 How to create a role for kubernettes and...
4,yn2eWCt,"[{'from': 'human', 'value': 'how do I add mult...",how do I add multiple new columns in m for pow...,[how do I add multiple new columns in m for po...,en,[how do I add multiple new columns in m for po...


In [11]:
# relevant conversations
# GwO8NfG
# Nf7eXj3

In [ ]:
corpus = df["text"]

In [ ]:
def preprocess_text(text):
    text = str(text)
    # remove angle brackets and all characters inside angle bracket
    # a=re.compile("<.*?>")
    # text = a.sub('', text)
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # convert to lowercase
    text = text.lower()
    # tokenize the text
    tokens = word_tokenize(text)
    # remove stopwords
    # tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens

# preprocess all text
preprocessed_text = [' '.join(preprocess_text(document)) for document in corpus]

In [ ]:
df["preprocessed_text"] = preprocessed_text

In [ ]:
df.head()

In [ ]:
corpus = df["preprocessed_text"]

In [ ]:
# tokenize the documents
tokenized_documents = [doc.split() for doc in corpus]


In [ ]:
# create a dictionary representation of the documents
dictionary = Dictionary(tokenized_documents)

In [ ]:
# convert document into bag-of-words format
corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]

In [ ]:
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, random_state=42)


In [ ]:
topics = lda_model.print_topics()
for topic in topics:
    print(topic)